# 🔢 Interactive Power Analysis for T20 Batting Pairs
This notebook lets you simulate and analyze statistical power when comparing two batting strategies in T20 cricket.
You can interactively change the average runs, standard deviation, and maximum sample size.

In [ ]:
# Install ipywidgets if not already installed
!pip install ipywidgets

In [ ]:
# Import libraries
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from statsmodels.stats.power import TTestIndPower
from ipywidgets import interact, FloatSlider, IntSlider
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Simulated power function
def simulate_power(n, mean1, mean2, sd, reps=1000, alpha=0.05):
    count = 0
    for _ in range(reps):
        group1 = np.random.normal(mean1, sd, n)
        group2 = np.random.normal(mean2, sd, n)
        _, p = stats.ttest_ind(group1, group2)
        if p < alpha:
            count += 1
    return count / reps

In [ ]:
# Interactive power curve plot
def interactive_power(mean1=39, mean2=50, sd=10, alpha=0.05, max_n=150):
    effect_size = (mean2 - mean1) / sd
    sample_sizes = np.arange(10, max_n + 1, 10)
    sim_powers = []
    analytical_powers = []
    power_analysis = TTestIndPower()
    for n in sample_sizes:
        sim_powers.append(simulate_power(n, mean1, mean2, sd, reps=500, alpha=alpha))
        analytical_powers.append(power_analysis.power(effect_size, nobs1=n, alpha=alpha, ratio=1.0))
    plt.figure(figsize=(10, 6))
    plt.plot(sample_sizes, sim_powers, 'o-', label='Simulated Power', color='blue')
    plt.plot(sample_sizes, analytical_powers, 's--', label='Analytical Power', color='green')
    plt.axhline(0.8, color='red', linestyle='--', label='80% Power Threshold')
    plt.title(f"Power Curve\nComparing Means: {mean1} vs {mean2} (Effect Size = {effect_size:.2f})")
    plt.xlabel('Sample Size per Group')
    plt.ylabel('Power')
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(True)
    plt.show()
    current_sim_power = simulate_power(30, mean1, mean2, sd, reps=500, alpha=alpha)
    current_analytical = power_analysis.power(effect_size, nobs1=30, alpha=alpha, ratio=1.0)
    print(f"Simulated Power @ n=30: {current_sim_power:.3f}")
    print(f"Analytical Power @ n=30: {current_analytical:.3f}")

In [ ]:
# Interactive widget
interact(
    interactive_power,
    mean1=FloatSlider(value=39, min=30, max=60, step=1, description='Mean Pair A'),
    mean2=FloatSlider(value=50, min=30, max=60, step=1, description='Mean Pair B'),
    sd=FloatSlider(value=10, min=5, max=20, step=1, description='Standard Dev'),
    max_n=IntSlider(value=150, min=30, max=300, step=10, description='Max Sample Size')
)